In [1]:
from selenium import webdriver
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import  WebDriverWait
import pandas as pd 
import time
import re
import os
import requests

In [3]:
def scrape(path):

    # to avoid windows to close again and again we make use of headless
    options = ChromeOptions()
    options.headless=True
    # make use of chrome for scraping
    service = Service(r"D:\GS_WebScarping\project_1\chromedriver.exe")
    # create a driver using chrome
    driver = Chrome(service=service, options=options)
    # run the driver through url
    driver.get(path)

# =====================================================================================
    # get the login icon and click on it 
# =====================================================================================
    try:
        # get the login button
        login = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".acc_button_alt")))
        if login:
            driver.execute_script("arguments[0].scrollIntoView();", login)
            login.click()
        else:
            pass

    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # complete logiin info
# =====================================================================================

    # get the username tab
    try:
        provided_u_name = "sultanmirza0501@gmail.com"
        user_name = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID,"Username")))   
        user_name.send_keys(provided_u_name)
    except Exception as e:
        print(f"No username tab found and the error is {e}")

    # get password tab
    try:
        provided_pass = "Muhssan7865"
        password = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, "Password")))
        password.send_keys(provided_pass)
    except Exception as e:
        print(f"No password tab found and the error is {e}")

    # get the login tab to enter the username and password
    try:
        # login_tab_css = "submit"
        login_tab = driver.find_element(By.XPATH,  "//input[@type='submit' and @value='Login']")
        driver.execute_script("arguments[0].scrollIntoView();", login_tab)
        login_tab.click()
        for i in range(2):
            driver.back()
    except Exception as e:
        print(f"No login tab found and the error is {e}")

# =====================================================================================
    # handle cookies
# =====================================================================================
    try:
        cookie = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.ID, "onetrust-reject-all-handler")))
        if cookie:
            cookie.click()
        else:
            pass
    except Exception as e:
        print("No cookies tab found and the error is {e}")

# =====================================================================================
    # get only number
# =====================================================================================
    try:
        import time
        time.sleep(2)
        num_cars = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".d-md-flex span"))).text
        cars_list = num_cars.split(" ")
        cars =  int(cars_list[1]) # get only number
        print(f"Number {cars} of cars found")
    except Exception as e:
        print(f"No number of cars found and error is {e}")


# # get the first car page link
# try:
#     first_car_page = driver.find_element(By.CSS_SELECTOR, "a.vehicle-card-link")
#     if first_car_page:
#         first_car_page_link = first_car_page.get_attribute("href")
        

   

# =====================================================================================
    # get the car link and click on it to proceed to main car information page
# =====================================================================================
    try:
        link = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".vehicle-card-header")))
        if link:
        # now we have to click on the link to get the main car info page
            driver.execute_script("arguments[0].scrollIntoView();", link)
            link.click()
        else:
            pass
    except Exception as e:
        print(f"No link found and the error is {e}")
# =====================================================================================  


# =====================================================================================
    # loop around the number of images present
# =====================================================================================
    results = [] # to append the dict values later on
    for i in range(cars):# we will make use of number of pages here

    # ===================================================================================== 
        # get the name of the car
    # =====================================================================================
        details = {}
        
        try:
            # car_name_css = ".row h2"
            car_name = driver.find_element(By.CSS_SELECTOR, ".row h2")
            if car_name:
                details["Car_name"] = car_name.text
            else:
                details["Car_name"] = "No car name for this car found"

        except Exception as e:
            print(f"No car name found and the error is {e}")
        
    # ===================================================================================== 
        # extract the car images 
    # =====================================================================================
        try:
            images=[]
            # import time
            # time.sleep(2)
            imgs = driver.find_elements(By.CSS_SELECTOR, "li.slide img")

            for img in imgs:
                images.append(img.get_attribute("src"))

            imgs_str = ", ".join(images)
            details['Images'] = imgs_str

        except Exception as e:
            print(f"No images found and the error is {e}")

    # =====================================================================================         
        # get the lot time, date, auction and center fields
    # =====================================================================================
        try:
            # container_selector = ".ab-catelogue-vehicle-details-auction-details"
            container = driver.find_element(By.CSS_SELECTOR, ".ab-catelogue-vehicle-details-auction-details")

            # Extract Auction Availability
            auction_availability = container.find_element(By.TAG_NAME, "p").text
            if auction_availability:
                details["Auction_Availability"] = auction_availability
            else:
                details["Auction_Availability"] = "No auction availability found"

            # Extract Lot, Time, and Date
            lot_time_date = container.find_elements(By.CSS_SELECTOR, "h3 span")
            if lot_time_date:
                lot = lot_time_date[0].text.replace("Lot: ", "")  # Removing 'Lot: ' prefix
                time = lot_time_date[1].text
                date = lot_time_date[2].text
                details['Lot'] = lot
                details['Time']= time
                details['Date'] = date
            else:
                details['Lot'] = "No lot number found"
                details['Time']= "No time found"
                details['Date'] = "No date found"


            # Extract Center
            center = container.find_element(By.CSS_SELECTOR, "h3:nth-of-type(2) span").text
            if center:
                details["Center"] = center
            else:
                details['Center'] = "No center found"

        except Exception as e:
            print(f"Nothing found and the error is {e}")
        
    # =====================================================================================
        # get the vehicle details
    # =====================================================================================
        try:
            # click on vehicle tab
            veh_tab = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "VEHICLE")))
            if veh_tab:
                driver.execute_script("arguments[0].scrollIntoView();", veh_tab)
                veh_tab.click()

                base_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tab-content")))
                
                # Locate the table inside the card
                table_selector = ".tab-pane.active .table tbody tr"
                rows = base_card.find_elements(By.CSS_SELECTOR, table_selector)

                if rows:
                    for row in rows:
                        # Find all <td> elements in the row
                        tds = row.find_elements(By.TAG_NAME, "td")
                        if len(tds) == 2:  # Ensure there are exactly two <td> elements
                            label = tds[0].text.strip()
                            value = tds[1].text.strip()
                            details[label] = value
                else:
                    print("No rows found")
            else:
                print("No vehicle tab found")
        except Exception as e:
            print(f"Novehcile info found and the error is {e}")

    # =====================================================================================
        # get the valuations
    # =====================================================================================
        try:

            valuation = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "VALUATIONS")))
            if valuation:
                driver.execute_script("arguments[0].scrollIntoView();", valuation)
                valuation.click()
                import time
                time.sleep(1)

                val_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tab-content")))
                if val_card:
                    val_rows = val_card.find_elements(By.CSS_SELECTOR, ".tab-pane.active .table tbody tr")
                    if val_rows:         
                        for val_row in val_rows:
                            # Find all <td> elements in the row
                            val_tds = val_row.find_elements(By.TAG_NAME, "td")
                            if len(tds) == 2:  # Ensure there are exactly two <td> elements
                                val_label = val_tds[0].text.strip()
                                val_value = val_tds[1].text.strip()
                                details[val_label] = val_value
                    else:
                        print("No value rows found inside valaution card")
                else:
                    print("No valuation card found")
            else:
                print("No valaution button found")

        except Exception as e:
            print(f"No valaution button found and the error is {e}")
        
    # =====================================================================================
        # get the INSPECTION REPORT
    # =====================================================================================
        try:

            inspection = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, "INSPECTION REPORT")))
            if inspection:
                driver.execute_script("arguments[0].scrollIntoView();", inspection)
                inspection.click()
                import time
                time.sleep(1)
            
                # Now extract the fields inside the inspection report
                inspec_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tab-content")))
            
            # =====================================================================================
                # get the icon image link at the side of the name of the car
            # =====================================================================================
                try:
                    # inspec_rep_css = WebDriverWait(driver, 60).until(EC.presence_of_element_located((By.XPATH, "//a[text()='Download the inspection report']")))
                    inspec_rep_css = driver.find_element(By.XPATH, "//a[text()='Download the inspection report']")
                    if inspec_rep_css:
                        inspec_link = inspec_rep_css.get_attribute('href')
                        details["Inspection_report"] =  inspec_link
                    else:
                        details["Inspection_report"] = "No inspection report found"
                except Exception as e:
                    print("No car inspection report tab found")

            # ===================================================================================== 
                # get the icon image link at the side of the name of the car
            # =====================================================================================
                try:
                    icon_image_link = inspec_card.find_element(By.XPATH, "//a[contains(@href, '.pdf')]")

                    if icon_image_link:
                        icon_href = icon_image_link.get_attribute("href")
                        details['Icon_image_link'] = icon_href
                    else:
                        details['Icon_image_link'] = "N/A"
                except Exception as e:
                    print(f"No icon image tab found")

                
                # Locate the table inside the card (looking specifically for the 'Tyres' table)
                tyres_table = inspec_card.find_element(By.XPATH, "//h3[contains(text(), 'Tyres')]//following::table[1]")
                tyres_rows = tyres_table.find_elements(By.TAG_NAME, "tr")

                # Extract other fields as before (standard inspection fields)
                inspec_table_selector = ".tab-pane.active .table tbody tr"
                inspec_rows = inspec_card.find_elements(By.CSS_SELECTOR, inspec_table_selector)
                
                for inspec_row in inspec_rows:
                    inspec_tds = inspec_row.find_elements(By.TAG_NAME, "td")
                    
                    # If there are exactly two <td> elements, process the label-value pair
                    if len(inspec_tds) == 2:
                        inspec_label = inspec_tds[0].text.strip()
                        inspec_value = inspec_tds[1].text.strip()
                        details[inspec_label] = inspec_value
                # Lists to store values for Location, Manufacturers, and Depth
                location_values = []
                manufacturers_values = []
                depth_values = []

                # Process each row inside the Tyres table
                for row in tyres_rows:
                    tds = row.find_elements(By.TAG_NAME, "td")
                    if len(tds) == 3:  # We expect three columns for Location, Manufacturers, Depth
                        location_values.append(tds[0].text.strip())
                        manufacturers_values.append(tds[1].text.strip())
                        depth_values.append(tds[2].text.strip())

                        details['Location'] = ", ".join(location_values)
                        details['Manufacturers'] = ", ".join(manufacturers_values)
                        details['Depth'] = ", ".join(depth_values)
            else:
                print("No inspection button found")
        except Exception as e:
            print(f"No inspection button found and error is {e}")
        
    # =====================================================================================
        # get the SPECIFICATION
    # =====================================================================================
        try:
            # get the button and then click on it
            spec_text = "SPECIFICATION"
            specs = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, spec_text)))
            if specs:
                driver.execute_script("arguments[0].scrollIntoView();", specs)
                specs.click()
                import time
                time.sleep(1)

                # now extract the fields inside valautions
                spec_card_class = "tab-content"
                spec_card = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, spec_card_class)))
                
                # Locate the table inside the card
                spec_table_selector = ".tab-pane.active .table tbody tr"
                spec_rows = spec_card.find_elements(By.CSS_SELECTOR, spec_table_selector)
                
                td_texts = [td.text for td in spec_rows if td.text.strip() != ""]
                details['Specifications']=td_texts
            else:
                print("No specs button found")
        except Exception as e:
            print(f"No specification button found and error is {e}")


    # =====================================================================================
        # Extract the image source URL
    # =====================================================================================
        try:
            damage_text = "DAMAGE REPORT"
            damages = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.LINK_TEXT, damage_text)))
            if damages:
                driver.execute_script("arguments[0].scrollIntoView();", damages)
                damages.click()
                import time
                time.sleep(1)

                # Initialize list for storing damaged image URLs and key-value pairs
                damaged_list = []
                details_list = []

                # Extract all image elements with the class 'magnifier-image'
                img_elements = driver.find_elements(By.CLASS_NAME, 'magnifier-image')
                for img in img_elements:
                    damaged_list.append(img.get_attribute("src"))
                
                # Assuming there's only one image to focus on for now
                damaged_urls = ", ".join(damaged_list)
                details["Damaged_image"] = damaged_urls

                # Extract key-value pairs from the damage table
                damage_rows = driver.find_elements(By.CSS_SELECTOR, '.tab-pane.active .table tbody tr')

                for damage_row in damage_rows:
                    # Get the first and second <td> elements
                    columns = damage_row.find_elements(By.TAG_NAME, 'td')
                    if len(columns) >= 2:  # Ensure there are exactly two columns
                        column_name = columns[0].text.strip()
                        column_value = columns[1].text.strip()
                        details_list.append({column_name: column_value})

                details["Damage_details"] = details_list  # Add to the 'details' dictionary
            else:
                print("No damage button found")
        except Exception as e:
            print(f"No damage button found and error is {e}")
    # =====================================================================================
    # =====================================================================================

        # append the in the results 
        results.append(details)

    # =====================================================================================
        # get the next button
    # =====================================================================================
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "a.vehicle-details-btn-next")
            if next_button:
                # get the link
                car_page_link = next_button.get_attribute("href")
                details['Car_link']= car_page_link
                next_button.click()
            else:
                details['Car_link'] = "N/A"
                
                break
        except Exception as e:
            print("No next button found hence no more cars are available")

# =====================================================================================
    # create a dataframe 
    df = pd.DataFrame.from_dict(results)
    df.to_csv("data2.csv")

    driver.quit()

# call the function
path = "https://catalogue.astonbarclay.net/vehicle-search/?search=true&sites=1&sales=14734"
scrape(path)

Number 3 of cars found
No next button found hence no more cars are available


### here the car_link column contains the list of the car page links, the first link belongs to the second car and rest follow this series, the last row will have the second last link

Number 8 of cars found
No car inspection report tab found
No icon image tab found
No next button found hence no more cars are available


In [121]:
df = pd.read_csv("data2.csv")
df['Car_link'][0]

'https://catalogue.astonbarclay.net/details/catalogue/108063371'

# Function to download the original and damage images

In [13]:
reg_img = df[['REG', "Images", "Damaged_image"]]

from urllib.parse import urlparse, urljoin
def download_images(data, main_folder="Images"):
    # Create the main folder if it doesn't exist
    os.makedirs(main_folder, exist_ok=True)
    
    # Loop through every row to get the info
    for index, row in data.iterrows():
        reg_no = row["REG"]  # Separate reg nums
        
        # Split image URLs by commas for both normal and damaged images
        image_urls = row["Images"].split(", ") if pd.notna(row["Images"]) else []
        damaged_urls = row["Damaged_image"].split(", ") if pd.notna(row["Damaged_image"]) else []
        
        # Create a subfolder for the car registration number
        reg_folder = os.path.join(main_folder, reg_no)  # Combine the main folder name and the reg num like Image/reg_no
        os.makedirs(reg_folder, exist_ok=True)  # Create the subfolder
        
        # Create a subfolder for images inside the registration folder
        images_folder = os.path.join(reg_folder, "Original_images")  # Folder for regular images
        os.makedirs(images_folder, exist_ok=True)  # Ensure the "Images" folder exists

        # Create a subfolder for damaged images inside the registration folder
        damaged_folder = os.path.join(reg_folder, "Damaged_images")  # Folder for damaged images
        os.makedirs(damaged_folder, exist_ok=True)  # Ensure the "Damaged" folder exists

        # Download regular images
        for idx, url in enumerate(image_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")):  # Check if the URL starts with 'http' or 'https'
                url = urljoin("https://", url)  # Add 'https://' if missing

            # Parse the URL
            parsed_url = urlparse(url)

            # Check if the parsed URL is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue
            
            # Try downloading the image
            try:
                # Download the image
                response = requests.get(url, stream=True)
                response.raise_for_status()  # Raises an exception if the HTTP request fails

                # Extract the file name from the URL
                file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

                # Check for valid file extensions
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"image_{idx + 1}.jpg"  # Assign a default name if extension is invalid
                
                # Construct the full path for the file inside the "Images" folder
                full_file_name = os.path.join(images_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

        # Download damaged images
        for idx, url in enumerate(damaged_urls):
            url = url.strip()  # Remove extra spaces
            if not url.startswith(("http://", "https://")):
                url = urljoin("https://", url)  # Add 'https://' if missing

            # Parse the URL
            parsed_url = urlparse(url)

            # Check if the parsed URL is valid
            if not parsed_url.scheme or not parsed_url.netloc:
                print(f"Invalid URL skipped: {url}")
                continue
            
            # Try downloading the image
            try:
                # Download the image
                response = requests.get(url, stream=True)
                response.raise_for_status()  # Raises an exception if the HTTP request fails

                # Extract the file name from the URL
                file_name = os.path.basename(parsed_url.path) or f"damaged_{idx + 1}.jpg"

                # Check for valid file extensions
                file_extension = file_name.split(".")[-1]
                if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
                    file_name = f"damaged_{idx + 1}.jpg"  # Assign a default name if extension is invalid
                
                # Construct the full path for the file inside the "Damaged" folder
                full_file_name = os.path.join(damaged_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
                # Save the image
                with open(full_file_name, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
                
                print(f"Downloaded: {full_file_name}")
            except Exception as e:
                print(f"Failed to download {url} for {reg_no}: {e}")

# Assuming `reg_img` contains the image data with "REG", "Images", and "Damaged_image"
download_images(reg_img)


Downloaded: Images\BL17CMU\Original_images\BL17CMU_1_659a7495-cfdc-447b-aec8-3bd945ccdab7_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_2_0aba4f81-e5c5-4c8e-b1f5-b4f8489c5d22_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_3_1d0b624a-a0c6-4fc6-9a23-852eeed01f85_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_4_44d6b676-5ab4-4bcc-b208-8dadf0ab037c_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_5_8a048c20-24e7-4474-8f75-815bbe2e20c5_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_6_3538558f-aaaf-4263-b641-2bdf4c4cc06f_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_7_9e4cdb45-3690-44fa-aec8-7f303962b57d_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_8_bdfa6045-1857-46f6-a77e-aab3c12ddd6c_removebg.jpg
Downloaded: Images\BL17CMU\Original_images\BL17CMU_9_281d31e3-03dd-4a56-b24f-8c171b9c7986_removebg.jpg
Downloaded: Images\BL17CMU\Damaged_images\BL17CMU_1_7cdcc779-a594-41c8-a3

In [ ]:
# from urllib.parse import urlparse, urljoin
# # urlparse: Parses a URL into components (scheme, netloc, path, etc.), making it easy to validate or extract information from the URL.
# # urljoin: Joins a base URL with a relative path to form a complete URL.

# def download_images(data, main_folder="Images"): # here the main folder is Images
    
#     # Create the main folder if it doesn't exist
#     os.makedirs(main_folder, exist_ok=True)
    
#     # loop around every row to get the info
#     for index, row in data.iterrows():
#         reg_no = row["REG"] # separate reg nums
#         image_urls = row["Images"].split(", ")  # Split image URLs by comma if multiple
        
#         # Create a subfolder for the car registration number
#         reg_folder = os.path.join(main_folder, reg_no) # combine the main folder name and the reg num like Image/reg_no
#         os.makedirs(reg_folder, exist_ok=True) # create the subfolder of the type mention in the above line of code
        
#         # loop around the urls of the current row and also save index for further use
#         for idx, url in enumerate(image_urls):
#             url = url.strip()  # Remove extra spaces
#             if not url.startswith(("http://", "https://")): # check if the url does not start with the values in the bracket
#                 url = urljoin("https://", url) # sets the urls starting from https://.....
            
#             # parse the url
#             parsed_url = urlparse(url)

#             # check if the parsed url is valid
#             if not parsed_url.scheme or not parsed_url.netloc:
#                 print(f"Invalid URL skipped: {url}") # incase some urls are incorret and are not loaded for downloading 
#                 continue
            
#             # try downloading images
#             try:
#                 # Download the image
#                 response = requests.get(url, stream=True) # send the url for downloading
#                 response.raise_for_status() # Raises an exception if the HTTP request fails (e.g., 404 or 500).
                
#                 # Extracts the file name from the URL path (e.g., image.jpg from http://example.com/image.jpg).
#                 # If no file name is found, assigns a default name based on the index.
#                 file_name = os.path.basename(parsed_url.path) or f"image_{idx + 1}.jpg"

#                 # Extracts the file extension (e.g., jpg). from the last index value
#                 file_extension = file_name.split(".")[-1]
                
#                 # Ensure the file has a valid extension
#                 if file_extension not in ["jpg", "jpeg", "png", "gif", "bmp", "webp"]:
#                     file_name = f"image_{idx + 1}.jpg" # set the extension if the extension is not in the above list
                
#                 # Construct the full path for the file
#                 full_file_name = os.path.join(reg_folder, f"{reg_no}_{idx + 1}_{file_name}")
                
#                 # Save the image
#                 with open(full_file_name, 'wb') as f:
#                     for chunk in response.iter_content(1024): # Reads the response in 1KB chunks to save memory.
#                         f.write(chunk)
                
#                 print(f"Downloaded: {full_file_name}")
#             except Exception as e:
#                 print(f"Failed to download {url} for {reg_no}: {e}")
# # Call the function
# download_images(reg_img)
